In [ ]:
#=======================================================================================
# sentence-bert와 Faiss 라이브러리를 이용하여 search 하는 예시임
# => Faiss는 facebook에서 만든 vector 유사도를 측정하는 라이브러리로, 
# 기존 numpy 나 scikit-learn 에서 제공하는 cosine similarity 보다 강력하며, GPU도 지원한다.
#
# 참고 
# 번역본 : https://ichi.pro/ko/transformers-mich-faissleul-sayonghayeo-simaentig-geomsaeg-enjin-eul-guchughaneun-bangbeob-242711337083112
# (원본 : https://medium.com/towards-data-science/how-to-build-a-semantic-search-engine-with-transformers-and-faiss-dcbea307a0e8)
#
# 설치
# Faiss는 설치는 https://github.com/facebookresearch/faiss/blob/main/INSTALL.md 참조
# GPU 버전 : $ conda install -c conda-forge faiss-gpu
# 
#=======================================================================================

import faiss
import numpy as np
import pandas as pd
import time

from os import sys
sys.path.append('../../')
from myutils import GPU_info, seed_everything

device = GPU_info()
seed_everything(111)

In [ ]:
# sentence bert 로딩
from sentence_transformers import SentenceTransformer
from sentence_transformers import models

smodel_path = "bongsoo/albert-small-kor-sbert-v1.1"

# 임베딩 벡터 폴링 모드 선택 (*아래값중 문자열로 입력함, 기본=mean)
# mean=단어 평균, max=최대값, cls=문장, 
#['mean', 'max', 'cls', 'weightedmean', 'lasttoken']
pooling_mode = 'cls'

word_embedding_model = models.Transformer(smodel_path, max_seq_length=256, do_lower_case=True, tokenizer_name_or_path=smodel_path)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode=pooling_mode)  
smodel = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    
#smodel = SentenceTransformer(smodel_path)
smodel.to(device)


In [ ]:
# list 데이터들을 dataframe으로 만듬
data = [
        [1, '독도 해역 헬기 추락사고가 발생한 지 열하루가 지났지만 실종자 추가 발견 소식은 들려오지 않고 있다. 헬기 동체 잔해물과 부유물 등은 발견되고 있지만 정작 실종자들은 발견하지 못해 수색이 장기화될 것이라는 우려가 현실이 될 조짐을 보여 실종자 가족들의 애를 태우고 있다.범정부현장수습지원단(지원단)은 10일 오전 10시 브리핑에서 이날 오전까지 독도해역 수색 결과 4점의 부유물을 추가 발견, 인양했다고 밝혔다'], 
        [11,'대구시는 11일 내년 정부 예산 3조1330억원을 확보해 전년보다 2%(611억원) 늘어났다고 밝혔다.그러나 올해 국비 증액 규모(1817억원)와 비교해 절반 수준에 그치며, 물산업클러스터 r&d 사업(200억), 국립청소년진로직업체험 수련원 건립 등은 한푼도 반영되지 않았다.'],
        [21,'자유한국당 (이름) 의원이 "공수처는 정권 유지를 위한 수단으로, 공수처가 있었으면 조국 수사를 못했을 것"이라고 주장했다.최 의원은 18일 대구 호텔수성에서 열린 대구 경북중견언론인모임 <아시아포럼21> 토론회에서 "검찰은 정권 말기가 되면 정권에 칼을 들이댔다. 공수처는 검찰의 칼끝을 회피하기 위한 것"이라며 이같이 말했다'],
        [2,'27일 오전 천연기념물(201-2호)이며 멸종위기 2급인 큰고니 떼가 경북 포항시 북구 흥해읍 샛강에 날아들었다.이날 관측된 큰고니는 어미 8마리와 새끼 3마리다.앞서 지난 19일 큰고니 8마리가 올들어 처음으로 관측됐다.큰고니들은 포항시 흥해읍 샛강에서 수초 등을 먹고 휴식한 후 대구 안심습지와 낙동강 하구 등지로 날아갈 것으로 보인다'],
        [3,'3일 대구와 경북지역은 구름이 많겠으며, 동해안과 북부 내륙에는 비가 내릴 것으로 예상된다.대구기상청에 따르면 중국 북동지방에서 남하하는 고기압의 가장자리에 들어 구름이 많겠으며, 경북 북부 내륙은 북쪽을 지나는 약한 기압골의 영향을 받아 오후부터 비가 약하게 내리거나 빗방울이 떨어지겠다.'],
        [31,'각급 정부기관의 통신망을 관리할 국가정보자원관리원 대구센터가 31일 대구 동구 도학동에서 착공했다.4312억원을 투입해 2021년 8월 준공 예정인 대구센터는 86개 기관의 서비와 장비를 운용하게 된다.정부통합전산센터에서 명칭이 변경된 국가정보자원관리원은 현재 대전본원과 광주센터를 운영 중이다.대구센터는 급변하는 행정환경과 4차 산업혁명 시대에 맞춰 클라우드, 빅데이터 등 신기술이 접목된 지능형 전산센터로 구축된다'],
        [4,'26일 오후 11시44분쯤 대구 달서구 신당동의 한 아파트에서 원인 모를 불이 나 주민 1명이 다치고 수십명이 대피했다.27일 소방당국에 따르면 신고를 받고 소방차 24대와 소방대원 74명이 출동해 20분만에 불길을 잡았다.이 불로 아파트 2층에 사는 a씨(53)가 온 몸에 화상을 입어 병원으로 이송됐으며, 연기에 놀란 주민 50여명이 대피하는 소동을 빚었다.']
       ]
df = pd.DataFrame(data, columns=['uid','text'])
print(df['text'].values)

In [ ]:
#!pip install umap-learn  # umap 설치
import umap

def embeddingforfaiss(df):
    # embedding 생성(인코딩)

    start = time.time()
    embeddings = smodel.encode(df.text.to_list(), show_progress_bar=True, convert_to_tensor=False)

    #tembeddings = np.transpose(embeddings, (1, 0))  # 첫 번째 축과 두 번째 축을 바꿈(768, 7) 식으로 바꿔야함.
    #print(tembeddings.shape)
    
    # UMAP을 사용하여 임베딩 벡터의 차원을 줄임
    #umap_embeddings = umap.UMAP(n_neighbors=2, n_components=128, metric='cosine').fit_transform(tembeddings)
    #print(umap_embeddings.shape)

    #float32 로 embeddings 타입 변경
    embeddings = np.array([embedding for embedding in embeddings]).astype("float32")
    #print(uembeddings.shape)

    # instance index 생성
    index = faiss.IndexFlatL2(embeddings.shape[1])

    # id를 매핑 시켜줌 => 이때 idtype은 반드시 int64 type이어야 함.
    index = faiss.IndexIDMap2(index)
    index.add_with_ids(embeddings, df.uid.values)

    print(f'인코딩 시간 : {time.time()-start:.4f}')
    
    return index

In [ ]:
#dataframe 인코딩
index = embeddingforfaiss(df)

In [ ]:
# 여러문장 쿼리 해봄
user_query = ["나는 오늘 여행갈거라서 너무 행복하다", 
             "나는 오늘 너무 슬프다"]

vector = smodel.encode(user_query)
distance, idx = index.search(np.array(vector).astype("float32"), k=4)

print(distance)
print(idx)
#print([list(df[df.idx == num]['text']) for num in idx[0]])
#print([list(df[df.idx == num]['text']) for num in idx[1]])

for i, query in enumerate(user_query):
    print(f'Q: {query}')
    count = 0
    for num in idx[i]:
        text = df[df.uid == num]['text'].values
        print(f'{num}, {text[0]}({distance[i][count]:.3f})')
        count += 1
    print('\n')
        


In [ ]:
# 데이터 추가
# list 데이터들을 dataframe으로 만듬
data1 = [
        [5, '정말 감동적인 영화를 보았다'], 
        [51,'드디어 오늘 학교 간다'],
        [61,'레스토랑에서 맛있게 점심을 먹었다'],
        [6,'회사가는 출근길이 너무 막힌다'],
        [7,'6시 이후에는 퇴근 길이 너무 막힌다'],
        [71,'오늘 손홍민이 하는 국가대표 축구 경기를 봤다'],
        [8,'tv에서 드라마를 보면서 울었다']
       ]
df1 = pd.DataFrame(data1, columns=['uid','text'])
print(df1['text'].values)

In [ ]:
# 추가된 문장 embedding 생성(인코딩)->추가 
# - 인덱스 만드는 부분마 없음.
def embeddingadd(df, index):
    embeddings = smodel.encode(df1.text.to_list(), show_progress_bar=True, convert_to_tensor=False)
   
    #float32 로 embeddings 타입 변경
    embeddings = np.array([embedding for embedding in embeddings]).astype("float32")
    index.add_with_ids(embeddings, df1.uid.values)

In [ ]:
embeddingadd(df1, index)

In [ ]:
# 추가된 dataframe을 기존 df에 추가함
df = pd.concat([df, df1])
print(df)

In [ ]:
# 여러문장 쿼리 해봄
user_query = ["나는 오늘 여행갈거라서 너무 행복하다", 
             "나는 오늘 너무 슬프다"]

vector = smodel.encode(user_query)
distance, idx = index.search(np.array(vector).astype("float32"), k=8)

#print(distance)
#print(idx)
#print([list(df[df.uid == num]['text']) for num in idx[0]])
#print([list(df[df.uid == num]['text']) for num in idx[1]])


for i, query in enumerate(user_query):
    print(f'Q: {query}')
    count = 0
    for num in idx[i]:
        text = df[df.uid == num]['text'].values
        print(f'{num}, {text[0]}({distance[i][count]:.3f})')
        count += 1
    print('\n')
     

In [ ]:
# idx입력해서 vector값 구하기
index.reconstruct(1)

In [ ]:
# 인덱스 저장 => test.index 파일로 저장
faiss.write_index(index, "test.index")

In [ ]:
#불러오기
index2 = faiss.read_index("test.index")

index2.reconstruct(1)